In [12]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
import os
import cv2

import keras

from sklearn.metrics import confusion_matrix, classification_report 




In [13]:
image_dir=Path('preprocessed')


In [14]:
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df1 = pd.concat([filepaths, labels], axis=1)


In [15]:

# # Import label encoder
# from sklearn import preprocessing
  
# # label_encoder object knows how to understand word labels.
# label_encoder = preprocessing.LabelEncoder()
  
# # Encode labels in column 'species'.
# image_df1['Label']= label_encoder.fit_transform(image_df1['Label'])
  
# image_df1['Label'].unique()

In [16]:
image_df1

,Filepath,Label
0,preprocessed/alif/477.png,alif
1,preprocessed/alif/2976.png,alif
2,preprocessed/alif/4278.png,alif
3,preprocessed/alif/4216.png,alif
4,preprocessed/alif/791.png,alif
...,...,...
12579,preprocessed/ta/523.png,ta
12580,preprocessed/ta/3207.png,ta
12581,preprocessed/ta/2989.png,ta
12582,preprocessed/ta/1812.png,ta


In [17]:
image_df1.to_csv('image_labels.csv',index=False) 

In [18]:
train_df, test_df = train_test_split(image_df1, train_size=0.7, shuffle=True, random_state=7)

In [19]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [20]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(7, 22),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(7, 22),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(7, 22),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 7047 validated image filenames belonging to 3 classes.
Found 1761 validated image filenames belonging to 3 classes.
Found 3776 validated image filenames belonging to 3 classes.


In [21]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras import layers
from keras.utils import to_categorical, image_dataset_from_directory
from sklearn.metrics import confusion_matrix

model = tf.keras.models.Sequential()
inputs = tf.keras.Input(shape=(7, 22))
# Preprocessing 
model.add(layers.Resizing(height=128, width=128, interpolation='bilinear'))
model.add(layers.Rescaling(scale=1./255))

# Data Augmentation
#model.add(layers.RandomFlip(mode='horizontal_and_vertical'))
#model.add(layers.RandomRotation(factor=.2))

# First Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,5), input_shape = (224,224,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(5,1)))
model.add(layers.Activation('leaky_relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

# Second Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,1)))
model.add(layers.Activation('leaky_relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

# Third Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,1)))
model.add(layers.Activation('leaky_relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

# Feed forward
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Activation('leaky_relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(512))
model.add(layers.Activation('leaky_relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))


In [22]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
         tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
    ]
)

Epoch 1/5


2023-04-06 08:49:02.596261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


221/221 [==============================] - ETA: 0s - loss: 1.6579 - accuracy: 0.6474

2023-04-06 08:51:26.905575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


221/221 [==============================] - 158s 690ms/step - loss: 1.6579 - accuracy: 0.6474 - val_loss: 22.7006 - val_accuracy: 0.3294 - lr: 0.0010
Epoch 2/5
221/221 [==============================] - 123s 556ms/step - loss: 0.7931 - accuracy: 0.6824 - val_loss: 23.3610 - val_accuracy: 0.3294 - lr: 0.0010
Epoch 3/5
221/221 [==============================] - 121s 546ms/step - loss: 0.6432 - accuracy: 0.6956 - val_loss: 11.9288 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 4/5
221/221 [==============================] - 124s 560ms/step - loss: 0.5758 - accuracy: 0.7190 - val_loss: 0.9207 - val_accuracy: 0.7087 - lr: 0.0010
Epoch 5/5
221/221 [==============================] - 123s 557ms/step - loss: 0.5295 - accuracy: 0.7331 - val_loss: 0.9706 - val_accuracy: 0.7081 - lr: 0.0010


In [ ]:
model.save('Model.h5')
